# Synthetic turbulent magnetic field with CRPropa

In [ ]:
from crpropa import *

In [1]:
from pylab import *

## 1. Creation of field

### Properties of grid and field

In [ ]:
# Grid
spacing = 10 * pc
grid_number = 100

# Field
Brms = 5 * nG
lmin = 50 * pc 
lmax = 1000 * pc 
index = 5./3. # Kolmogorov index = 5/3, Kraichnan index = 3/2
random_seed = 42

# Creation of turbulent field
spectrum = SimpleTurbulenceSpectrum(Brms, lmin, lmax, index)
grid = GridProperties(Vector3d(0), grid_number, spacing)
BField = SimpleGridTurbulence(spectrum, grid, random_seed)

# Save field
dumpGridToTxt(BField.getGrid(), 'kolmogorov_field.txt')

### Check of field properties

In [ ]:
print('Lc = {:.1f} kpc'.format(BField.getCorrelationLength() / pc))  
print('sqrt(<B^2>) = {:.1f} nG'.format(BField.getBrms() / nG))   
print('<|B|> = {:.1f} nG'.format(BField.getMeanFieldStrength() / nG))

## 2. Diagnostic of field

### Extraction of field components of grid

In [ ]:
field = np.genfromtxt('kolmogorov_field.txt')

# Field components
Bx = field[:,0] / nG
By = field[:,1] / nG
Bz = field[:,2] / nG

# Field magntiudes
B = np.sqrt(Bx**2 + By**2 + Bz**2)

# Squared values of field
Bx_sqr = Bx**2
By_sqr = By**2
Bz_sqr = Bz**2
B_sqr = B**2

### Check of mean values

In [ ]:
Bx_mean = np.mean(Bx)
By_mean = np.mean(By)
Bz_mean = np.mean(Bz)

print('Mean Bx', Bx_mean)
print('Mean By', By_mean)
print('Mean Bz', Bz_mean)
print('Mean B', B_mean)

### Plotting distributions

In [ ]:
# Plot of field components
plt.figure(figsize = (15, 10))
#
plt.subplot(2, 2, 1)
counts, bins = np.histogram(Bx, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B_x$ [nG]', fontsize=17)
plt.grid()
#
plt.subplot(2, 2, 2)
counts, bins = np.histogram(By, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B_y$ [nG]', fontsize=17)
plt.grid()
#
plt.subplot(2, 2, 3)
counts, bins = np.histogram(Bz, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B_z$ [nG]', fontsize=17)
plt.grid()
#
plt.subplot(2, 2, 4)
counts, bins = np.histogram( B, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$\delta B [nG]$', fontsize=17)
plt.grid()
#
plt.savefig('mean_field_components_kolmogorov.pdf')

In [ ]:
# Plot of squared field components
plt.figure(figsize = (15, 10))
#
plt.subplot(2, 2, 1)
counts, bins = np.histogram(Bx_sqr, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B_x)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.subplot(2, 2, 2)
counts, bins = np.histogram(By_sqr, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B_y)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.subplot(2, 2, 3)
counts, bins = np.histogram(Bz_sqr, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B_z)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.subplot(2, 2, 4)
counts, bins = np.histogram( B**2, bins=50)
plt.hist(bins[:-1], bins, weights=counts)
plt.xlabel(r'$(\delta B)^2$', fontsize=17)
plt.yscale('log')
plt.grid()
#
plt.savefig('squared_field_components_kolmogorov.pdf')

## 3. Power spectrum

### FFT of field components

In [ ]:
# 3D FFT to go from x-space to k-space
B_ifft = np.array((Bx, By, Bz))
Bk = np.fft.fftn(B_ifft)
Bkx = Bk[0]
Bky = Bk[1]
Bkz = Bk[2]
Bk_total = np.sqrt( (Bkx.real**2 + Bkx.imag**2) + (Bky.real**2 + Bky.imag**2) + (Bkz.real**2 + Bkz.imag**2) )

### Computation of k-values of $\delta B^2 (k)$

In [ ]:
# Generating N_grids-possible discrete wave numbers
K = np.zeros(grid_number)
for i in range(0, grid_number):
    K[i] = (i / grid_number) - (i / (grid_number / 2.) )

# Computing vector with corresponding k values to each grid point
grid_number_z = int(np.floor(grid_number/2) + 1)
k = np.zeros(grid_number * grid_number * grid_number_z)
k_max = (spacing / pc) / (lmin / pc)
k_min = (spacing / pc) / (lmax / pc)
for ix in range(0, grid_number):
    for iy in range(0, grid_number):
        for iz in range(0, grid_number_z):
            i = ix * grid_number * grid_number_z + iy * grid_number_z + iz
            k[i] = np.sqrt(K[ix]**2 + K[iy]**2 + K[iz]**2)

range_k = len(k)
Bk_total = Bk_total[0:range_k]

# Binning of k-values and respective B^2
counts, bins = np.histogram(k, bins=50)
k_values = np.zeros(len(bins) - 1)
Bk_sqrd_values = np.zeros(len(bins) - 1)
for j in range(0, len(bins) - 1):
    k_values[j] = (bins[j] + bins[j+1]) / 2
    index_where = np.where(np.logical_and(k >= bins[j], k <= bins[j+1]))
    Bk_sqrd_values[j] = np.mean(Bk_total[index_where]**2)

### Plotting spectrum

In [ ]:
# Plotting power spectrum
plt.figure(figsize = (7, 5))
plt.plot(k_values, Bk_sqrd_values, color='blue', label=r'Test')
plt.loglog()
plt.ylim(1e9, 1e13)
plt.xlim(k_min, k_max)
plt.xlabel(r'k', fontsize=17)
plt.ylabel(r'$\delta B^2(k)$', fontsize=17)
plt.grid()
#plt.legend(fontsize=17)
plt.savefig('Plots/power_spectrum_kolmogorov.pdf')

### Corroborating spectrum slope

In [ ]:
# Check slope
index_slope = np.where(np.logical_and(k_values >= k_min, k_values <= k_max))
linear = linregress(np.log10(k_values[index_slope]), np.log10(Bk_sqrd_values[index_slope]))
print('Numerical slope of turbulence spectrum:', linear[0])
print('Theoretical slope of turbulence spectrum:', -index-2)